In [1]:
import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark import SparkConf, SparkContext

import package from libraries, here we only use SparkConf and SparkContext

In [2]:
conf = SparkConf().setMaster('local').setAppName('MatrixMultiplication')
sc = SparkContext(conf=conf)

In [3]:
 inputs = sc.textFile("500input.txt")

In [4]:
def mapper_j(x):
    p = x.split(',')
    if p[0] == 'M':
        return (p[2],[(p[0],p[1],p[3])])
    elif p[0] == 'N':
        return (p[1],[(p[0],p[2],p[3])])

map every elements into form (j,(M,i,Value)) (j,(N,k,Value))

In [5]:
def mapper_mul(x):
    len_M = int(len(x[1])/2)
    for i in range(len_M):
        for j in range(len_M):
            multi = float(x[1][i][2])*float(x[1][len_M+j][2])
            yield ((i,j), multi)

input x : (j,((M,i,Value) ... (N,k,Value))) for all i & j
for each element we create pairs ((i,k), value_M * value_N)

In [6]:
inputs_gb_j = inputs.map(mapper_j).reduceByKey(lambda p,q:p+q)

for each pair (j,(M,i,Value)) (j,(N,k,Value)) we reduce it into (j,((M,i,Value) ... (N,k,Value))) for all i & j

In [7]:
inputs_j = inputs_gb_j.collect()
inputs_j = sc.parallelize(inputs_j)

release it from RDD and put it back into RDD

In [8]:
muls = inputs_j.flatMap(mapper_mul).reduceByKey(lambda p,q : p+q)

map it by the mapper that we create before and collect and sum it by pair (i,k)

In [9]:
outputs = muls.collect()
f = open("Outputfile.txt",'w')
for lin in outputs:
    f.write(str(lin[0][0])+" "+str(lin[0][1])+" "+str(lin[1])+"\n")
f.close()

release it from RDD and save it to files

In [10]:
sc.stop()

end the spark 